# Decision Tree
- 분류 회귀 문제에 널리 사용하는 모델로 최종 결정에 다다르기까지 yes or no의 질문을 반복한다. 
- 하나의 데이터를 입력받으면 데이터를 위 질문부터 차례로 완료하면서 점점 세분화한다. 
- 한 분기마다 두 개의 변수 영역을 나누어 불순도나 불확실성이 감소하는 방향을 선택하여 트리를 형성하며 영역을 학습한다.

## 결정 트리의 프로세스
1. 루트 노드의 불순도를 계산
2. 나머지 속성에 대해 분할 후 자식 노드의 불순도를 계산
3. 각 속성에 대해 정보 획득을 계산하고, 최대가 되는 분기 조건을 찾아 분기
4. 모든 말단 노드의 불순도가 0이 될때까지 2-3번을 반복 
[참고](https://blog.naver.com/jaeyoon_95/222364521559)

## 의사결정트리의 장단점
(1) 장점
- 결정 트리의 장점은 결과를 해석하고 이해하기 쉽고, 자료를 가공할 필요가 거의 없다는 점입니다. 
- 수치 자료와 범주 자료 모두에 적용할 수 있고, 안정적이며 대규모 데이터 셋에서도 잘 동작합니다. 비교적 빠른 속도를 가집니다.  

(2)단점
- Overfitting이 발생할 가능성이 높습니다. 
- 연속형 변수값을 예측할 때 적당하지 않습니다. 
- 회귀모형에서 예측력이 매우 떨어집니다. 
- 트리 모형이 복잡하면 예측력이 저하됩니다. 
- 데이터 변형에 민감하여 안정적이지 않습니다.

#### 이러한 의사결정나무의 단점들을 해결하기 위해 랜덤포레스트라는 기법을 사용


# 랜덤 포레스트란?
- 기계학습에서 분류, 회귀분석 등에 사용되는 앙상블 학습 방법입니다. 
- 앙상블 기법에는 보팅, 배깅, 부스팅이 있는데 랜덤포레스트는 배깅을 기반으로 한 대표적 알고리즘입니다.
- '배깅' 방식으로 무작위로 데이터를 뽑아 바구니를 만들어 각각의 '의사결정 나무'를 만들어 '앙상블' 한다.
즉, 각각의 모델을 결합하여 하나의 모델을 만든다.

## 랜덤포레스트의 특징
- 비교적 빠른 수행속도
- 비교적 적은 하이퍼 파라미터
- 트리기반의 알고리즘   
- 과적합(Overfitting) 문제를 해소하여 정확성 향상  
[공식문서](https://scikit-learn.org/stable/mod)

### 랜덤포레스트 회귀는 반응변수가 연속형일 경우 사용한다. 
- 가격 또는 숫자를 예측할 경우 -> 회귀
- 회귀는 정확도를 확인하는 것이 아닌 설명력 , 오차 등을 확인하여 모델을 평가
- 예측된 결과와 실제값과 얼마나 차이가 나는지 확인하기 위해 RMSE를 본다.
- 오차가 적을수록 예측을 잘 했다고 할 수 있다.
- 결정계수(설명력)이 높을 수록 좋은 모델

### 랜덤포레스트의 활용

In [67]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [68]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [69]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#gridSearch를 통해 최적의 하이퍼파라미터를 찾고 cross_val_score로 모델 평가

df_train['요일'] = df_train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
df_test['요일'] = df_test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

x_train = df_train[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]
y1_train = df_train['중식계']
y2_train = df_train['석식계']

x_test = df_test[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]

파라미터 설명
- max_depth : 트리의 깊이 

- (트리의 깊이가 깊을 수록 정확성은 좋아지지만  너무 깊은면 과적합이 될 수 있어요!!)

- min_samples_leaf : 노드가 되려면 가지고 있어야할 최소 샘플 수

- (작을 수록 좋지만 너무 작으면 과적합이 될수도..)

- min_samples_split : 샘플이 최소한 몇개 이상이어야 split(하위 (잎) 노드로 분리) 결정 수

- n_estimators : 학습 수

In [74]:
#랜덤포레스트 파라미터 
params = {
    'n_estimators': (100,500),
    'max_depth' : (5,12),
    'min_samples_leaf':(8,18),
    'min_samples_split' : (8,20)
}

model1 = RandomForestRegressor(n_jobs=-1, random_state=42)
model2 = RandomForestRegressor(n_jobs=-1, random_state=42)

#최적의 조합 탐색
grid_cv1 = GridSearchCV(model1, cv=5, param_grid=params,scoring='neg_mean_absolute_error', n_jobs=-1)
grid_cv1.fit(x_train,y1_train)
grid_cv2 = GridSearchCV(model2, cv=5, param_grid=params,scoring='neg_mean_absolute_error', n_jobs=-1)
grid_cv2.fit(x_train,y2_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': (5, 12), 'min_samples_leaf': (8, 18),
                         'min_samples_split': (8, 20),
                         'n_estimators': (100, 500)},
             scoring='neg_mean_absolute_error')

In [75]:
print('중식계 최적 하이퍼파라미터 :',grid_cv1.best_params_)
print('중식계 최적 예측 정확도 :{0: 4f}'.format(grid_cv1.best_score_))
print()
print('석식계 최적 하이퍼파라미터 :',grid_cv2.best_params_)
print('석식계 최적 예측 정확도 : {0: 4f}'.format(grid_cv2.best_score_))

중식계 최적 하이퍼파라미터 : {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 500}
중식계 최적 예측 정확도 :-90.807419

석식계 최적 하이퍼파라미터 : {'max_depth': 5, 'min_samples_leaf': 18, 'min_samples_split': 8, 'n_estimators': 500}
석식계 최적 예측 정확도 : -85.167041


In [77]:
best_lunch = grid_cv1.best_estimator_
best_dinner = grid_cv2.best_estimator_

In [79]:
best_lunch,best_dinner

(RandomForestRegressor(max_depth=12, min_samples_leaf=8, min_samples_split=8,
                       n_estimators=500, n_jobs=-1, random_state=42),
 RandomForestRegressor(max_depth=5, min_samples_leaf=18, min_samples_split=8,
                       n_estimators=500, n_jobs=-1, random_state=42))

In [78]:
model_lunch= best_lunch
model_lunch.fit(x_train, y1_train)

model_dinner = best_dinner
model_dinner.fit(x_train, y2_train)

pred1 = model_lunch.predict(x_test)
pred2 = model_dinner.predict(x_test)

In [85]:
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10,shuffle=True,random_state=0)
score_l = cross_val_score(model_lunch,x_train,y1_train,cv=k_fold,scoring='neg_mean_absolute_error')
score_l = score_l.mean()
score_d = cross_val_score(model_dinner,x_train,y2_train,cv=k_fold,scoring='neg_mean_absolute_error')
score_d = score_d.mean()
print("score : {0:.5f}".format(score_l))
print("score : {0:.5f}".format(score_d))

score : -80.32857
score : -71.71775


모델평가지표

In [93]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#https://blog.naver.com/dbfladbfla12/221839023581 예제

모델링을 하려면 데이터가 있어야 하는 것은 필수  
train_test_split을 활용하여 데이터를 train,test로 나누어야 한다.
(위에서는 분리과정을 생략 -> 데이터 전처리 필요)  

random_state는 초기 난수값을 설정해주는 것으로 아무 숫자나 넣어도 된다.
(쓰는 이유는 결과값을 똑같이 재현하기 위해서)
  
train 데이터는 모델을 학습 하는데에 test 데이터는 학습한 모델을 검증 하는 데 사용된다.

모델을 빠른 파라미터 튜닝을 위해 gridsearch라는 최적화 기법을 사용
